In [10]:
!pip install langchain==0.0.309 --quiet
#!pip install openai==0.28.1 --quiet
!pip install openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 3.9 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
#import openai
import langchain
#from langchain import OpenAI
api_key =""
#openai.api_key = api_key

In [3]:
from langchain.memory import ChatMessageHistory

chat_history = ChatMessageHistory()

chat_history.add_user_message("Can you give me the solution for 10+10")
chat_history.add_ai_message("The Solution is 10+10=20")
chat_history.add_user_message("Thank You")

print(chat_history.messages)


[HumanMessage(content='Can you give me the solution for 10+10'), AIMessage(content='The Solution is 10+10=20'), HumanMessage(content='Thank You')]


# Building a ChatBot with LangChain
## Start with a chat llm object

In [4]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(openai_api_key=api_key)
#chat = OpenAI(openai_api_key=api_key)

chat([HumanMessage(content="Translate this sentence from English to French: I love programming.")])
#chat(prompt="Translate this sentence from English to French: I love programming.")

AIMessage(content="J'adore programmer.")

Note that the llm object has no memory. It is not "context aware":

In [5]:
chat([HumanMessage(content="Now in Spanish")])

AIMessage(content='Ahora en español. ¿En qué puedo ayudarte?')

## Next, use a ConversationChain to add memory/context
...and turn it into a ChatBot

In [6]:
from langchain.chains import ConversationChain

conversation = ConversationChain(llm=chat)
conversation.run("Translate this sentence from English to French: I love programming.")

"Je suis désolé, je ne suis pas capable de traduire des phrases de l'anglais vers le français pour le moment."

In [7]:
conversation.run("Now in Sparnish.")

'Lo siento, pero no puedo traducir frases del inglés al español en este momento.'

# Working with Memory (Chat Context)

In [8]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='hi!'), AIMessage(content='whats up?')]))

## Let's create a scripted conversation

In [9]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")
memory.chat_memory.add_user_message("Answer only in Croatian")
memory.chat_memory.add_ai_message("Understood. I will answer only in Croatian.")
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='hi!'), AIMessage(content='whats up?'), HumanMessage(content='Answer only in Croatian'), AIMessage(content='Understood. I will answer only in Croatian.')]))

## And then load it into a new ChatBot

In [10]:
from langchain.chains import ConversationChain

conversation = ConversationChain(llm=chat,memory=memory)
conversation.run("What is an LLM?")

'LLM je skraćenica za "Master of Laws" ili "magistar prava". To je napredni stupanj u pravu koji se može steći nakon završetka prvostupničkog studija prava. LLM programi omogućuju studentima da se specijaliziraju u određenim područjima prava i steknu dublje znanje i vještine u svojoj pravnoj karijeri.'

# Let's look at another approach
## Using a LLMChain, and passing in history through the prompt arg

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain #,ConversationChain

# LLM
llm = ChatOpenAI(openai_api_key=api_key)

# Prompt
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human. Answer only in Croatian."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human. Answer only in Croatian.
Human: hi

> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Bok! Kako mogu pomoći?')],
 'text': 'Bok! Kako mogu pomoći?'}